In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display
import time
import cvxpy as cp

In [ ]:
class CartPoleSystem:

    def __init__(self):
        self.gravity   = -10
        self.masscart  = 5.0
        self.masspole  = 1
        self.length    = 2
        self.drag = 1

    def ode(self, x, u):
        m = self.masspole
        M = self.masscart
        L = self.length
        d = self.drag
        g = self.gravity
        
        Sx = np.sin(x[2])
        Cx = np.cos(x[2])
        D = m*L*L*(M+m*(1-Cx**2))
        
        dx = np.zeros(4)

        dx[0] = x[1]
        dx[1] = (1/D)*((-(m**2)*(L**2)*g*Cx*Sx + m*(L**2)*(m*L*(x[3]**2)*Sx - d*x[1])) + m*L*L*u)
        dx[2] = x[3]
        dx[3] = (1/D)*((m+M)*m*g*L*Sx - m*L*Cx*(m*L*(x[3]**2)*Sx - d*x[1]) + m*L*Cx*u)

        return dx
    
    def euler(self, state, control, dt):
        """Discrete-time dynamics (Euler-integrated) of an inverted pendulum."""
        return state + dt * self.ode(state, control)
    
    def rk4(self, ode, state, control, dt):
        k1 = dt * ode(state, control)
        k2 = dt * ode(state + k1 / 2, control)
        k3 = dt * ode(state + k2 / 2, control)
        k4 = dt * ode(state + k3, control)
        return state + (k1 + 2 * k2 + 2 * k3 + k4) / 6

In [ ]:
def plot_traj(state_sequence): 
    """Input a N*4*1 matrix, will prinit trajectory as an animation"""
    N = len(state_sequence)

    for i in range(N):
        # pole
        plt.scatter(state_sequence[i,0]+CartPoleSystem().length*np.sin(state_sequence[i,2]),-CartPoleSystem().length*np.cos(state_sequence[i,2]),marker='o',label = 'upper_mass')
        # cart
        plt.scatter(state_sequence[i,0],0,marker='o', s = 70,label = 'cart')
        
        plt.grid()
        plt.legend()
        plt.xlim([-10, 10])
        plt.ylim([-10, 10])
        
        IPython.display.clear_output(wait=True)
        display(plt.gcf())
        time.sleep(0.01)
        plt.clf()  # Clear the current figure

In [ ]:
car = CartPoleSystem()

#generate random u, u \in R^1
N = 1000 #steps
u = np.zeros(N)
dt = 0.5

# initial state
x0 = np.array([0,0,np.deg2rad(100),0]) 
# x0 = np.array([-1,0,np.pi+0.1,0]) 

state = np.zeros((N+1,4))
state[0] = x0
for i in range(N):
    state[i+1] = car.rk4(car.ode,state[i],u[i],dt)

plot_traj(state)

### MPC


In [ ]:
# set up trajectory optimization problem but with initial state as a parameter that can be updated
def setup_trajopt(dis_func, Q, R, Qt, goal, MPC_horizon, dt):
    n = 4 #state dimention
    m = 1 #control dimention
    us = cp.Variable([MPC_horizon,m], name="controls")
    xs = cp.Variable([MPC_horizon+1,n], name="states")
    # initial state is a parameter. It can be updated before solving the problem
    initial_state = cp.Parameter(n, name="initial_state")
    
    objective = 0
    constraints = [] 

    for t in range(MPC_horizon):
        objective += cp.quad_form(xs[t,:]-goal,Q) + u[t]*u[t]*R
        constraints += [xs[t+1,:] == dis_func(xs[t,:],us[t,:],dt,cp),cp.abs(xs[t]) <= 20]
    objective += cp.quad_form(xs[-1]-goal,Qt)  
    constraints += [xs[0,:] == initial_state]


    problem = cp.Problem(cp.Minimize(objective), constraints)
    return problem, xs, us

# function to update the value of initial_state
def update_initial_state(problem, initial_state):
    problem.param_dict["initial_state"].project_and_assign(initial_state)
    return problem

In [ ]:
dt = 0.1
n = 4 #state_dimention
m = 1 #control_dimention

N_MPC_horizon = 10
N_total_horizon = 80

cart = CartPoleSystem()

A_dynamics = cart.get_Ad(dt)
B_dynamics = cart.get_Bd(dt)


Q = np.diag([0.5, 0.5, 10., 5.]) 
R = np.diag([0.1, 0.1])

MPC_states = np.zeros((N_total_horizon+1,n))   # size [time_horizon+1, n]
MPC_controls = np.zeros((N_total_horizon+1,m)) # size [time_horizon, m]

x0 = np.array([0,0,0.5*np.pi,0])
goal_state = np.array([0,0,0,0])

k = 0
MPC_states[0] = x0 
while k < N_total_horizon:
    prob, xs, us = setup_trajopt(cart.discrete_step,Q,R,Q,goal_state,N_MPC_horizon,dt)
    prob = update_initial_state(prob, MPC_states[k])
    prob.solve()

    print(k)
    print(xs.value)

    MPC_states[k+1] = xs.value[1]
    MPC_controls[k] = us.value[0]
    k += 1


In [ ]:
u = MPC_controls

# for i in range(N):
#     if (i%2 !=0):
#         u[i] = 0.1*(1+i)
#     else:
#         u[i] = 0.1*(-1-i)

# initial state
x0 = np.array([0,0,np.pi,0]) 

state = np.zeros((N_total_horizon+1,4))
state[0] = x0
for i in range(N_total_horizon):
    state[i+1] = cart.discrete_step(state[i],u[i],dt)

plot_traj(state)
